In [1]:
import pdf4llm
import json
from llama_index.core.node_parser import SentenceSplitter

md_read = pdf4llm.LlamaMarkdownReader()
data = md_read.load_data("Juego de tronos - Canción de hielo y fuego 1 (1) copy.pdf")


Successfully imported LlamaIndex
Processing Juego de tronos - Canción de hielo y fuego 1 (1) copy.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing Juego de tronos - Canción de hielo y fuego 1 (1) copy.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing Juego de tronos - Canción de hielo y fuego 1 (1) copy.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing Juego de tronos - Canción de hielo y fuego 1 (1) copy.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing Juego de tronos - Canción de hielo y fuego 1 (1) copy.pdf...
[                                        ] (0/1====================

In [2]:
def preprocess(text):
    text = text.strip()
    text = text.replace("_", "")
    text = text.replace("\n", " ")
    text = text.replace("  ", " ")
    text = text.replace("-----", "")
    return text


def save_json(data, file):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
        
def chunk_text(data, size, overlap):
    splitter = SentenceSplitter(
        chunk_size=size,
        chunk_overlap=overlap,
    )
    nodes = splitter.get_nodes_from_documents(data)
    return nodes

def process_nodes(nodes):
    chunks_sentences = []
    for id, node in enumerate(nodes):
        page = node.metadata['page']
        text = preprocess(node.text)
        chunk = {'chunk_id': id, 'page': page, 'preprocess_content': text, 'content': node.text}
        chunks_sentences.append(chunk)
    return chunks_sentences
        

In [38]:
### Creo distintos chunks del texto original
size = 1024
overlap = 100
chunks_1024 = chunk_text(data, size, overlap)
chunks_sentences = process_nodes(chunks_1024)
save_json(chunks_sentences, "jdt_chunks_sentences_1024.json")

In [39]:
### Creo distintos chunks del texto original
size = 514
overlap = 50
chunks_512 = chunk_text(data, size, overlap)
chunks_sentences = process_nodes(chunks_512)
save_json(chunks_sentences, "jdt_chunks_sentences_512.json")



In [3]:
### Creo distintos chunks del texto original
size = 256
overlap = 50
chunks_256 = chunk_text(data, size, overlap)
chunks_sentences = process_nodes(chunks_256)
save_json(chunks_sentences, "jdt_chunks_sentences_256.json")

### Disclaimer:

- Despues de probar algunas iteraciones con diferentes tamaños de chunks, el que creo que funcionó mejor fue el de 512. Esta version es la utilizada en la entrega. 
- Me quede sin tiempo de implementar un metodo de evaluación y asi tomar una decisión más acertada en terminos de accuracy, no obstante creo que en función de los objetivos de la POC, podría ser suficiente.  
- La unica limpieza sobre los datos fue la eliminación de espacios multiples y guiones. Me percaté un poco in media res de las tildes y las particularidades del español, pero incluiria una serie de funciones de normalización del texto en todos los stages del proceso. Que a fin de cuentas solo es más exhaustiva en el módulo del clasificador de dominio. 
- Además, hay un tema con el parseo de los pdfs y la division natural del libro en paginas. Creo que el parser se apega a esta división y corta de algun modo con el desarrollo 'humano' del proceso de lectura, quien al llegar al final de una pagina, es capaz de ligar la informacion con la pagina siguiente. Una punta para la que tirar.
